# 03 - SQL + Pandas Hybrid Dashboards

This notebook blends SQL views from the consolidated SQLite warehouse with pandas wrangling and Plotly dashboards to deliver daily compliance KPIs and workout-linked glucose insights.

### Notebook Roadmap
- Connect to `health_glucose.db` and surface available tables/views
- Compose reusable SQL helpers that feed pandas DataFrames
- Model daily time-in-range compliance and derivative metrics
- Quantify glucose shifts around workouts leveraging hybrid SQL+pandas
- Render shareable Plotly dashboards and export tidy artifacts

In [1]:
import sqlite3
from pathlib import Path
from datetime import datetime, timedelta
from typing import Optional, Tuple, Any

import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

pd.set_option("display.max_columns", None)
pd.set_option("display.precision", 2)
pd.options.plotting.backend = "plotly"

print("✅ Libraries loaded")
print(f"📊 pandas {pd.__version__}")

✅ Libraries loaded
📊 pandas 2.3.2


In [ ]:
project_root = Path("..").resolve()
db_path = project_root / "data" / "database" / "health_glucose.db"

if not db_path.exists():
    raise FileNotFoundError(f"Database not found at {db_path}")

print(f"✅ Database located: {db_path}")

✅ Database located: /Users/george/Library/Mobile Documents/com~apple~CloudDocs/Programming Projects/Apple-Health-DS/data/database/health_glucose.db


In [ ]:
def get_connection(db_file: Path) -> sqlite3.Connection:
    """Create a SQLite connection with Row factory for named access."""
    conn = sqlite3.connect(db_file)
    conn.row_factory = sqlite3.Row
    return conn


def run_query(query: str, params: Optional[tuple] = None) -> list[sqlite3.Row]:
    """Execute a SQL query and return sqlite3.Row results."""
    with get_connection(db_path) as conn:
        cursor = conn.execute(query, params or ())
        rows = cursor.fetchall()
    return rows


def read_query_df(query: str, params: Optional[tuple] = None) -> pd.DataFrame:
    """Execute a SQL query and return a pandas DataFrame."""
    with get_connection(db_path) as conn:
        df = pd.read_sql_query(query, conn, params=params)
    return df


def get_date_range(
    table: str, column: str
) -> Tuple[Optional[pd.Timestamp], Optional[pd.Timestamp]]:
    """Fetch min/max timestamps for the specified table column."""
    query = (
        f"SELECT MIN({column}) AS start_date, MAX({column}) AS end_date FROM {table}"
    )
    rows = run_query(query)
    if not rows:
        return None, None
    row = rows[0]
    start = pd.to_datetime(row["start_date"]) if row["start_date"] else None
    end = pd.to_datetime(row["end_date"]) if row["end_date"] else None
    return start, end


print("✅ Helper utilities registered")

✅ Helper utilities registered


In [ ]:
catalog_df = read_query_df(
    "SELECT name, type FROM sqlite_master WHERE type IN ('table', 'view') ORDER BY type, name;"
)
catalog_df

,name,type
0,apple_health_records,table
1,data_quality_log,table
2,glucose_readings,table
3,glucose_statistics,table
4,merged_health_data,table
5,sqlite_sequence,table
6,workout_records,table
7,daily_glucose_summary,view
8,hourly_glucose_patterns,view
9,workout_glucose_impact,view


In [ ]:
glucose_bounds = get_date_range("glucose_readings", "timestamp")
workout_bounds = get_date_range("workout_records", "start_date")

summary = pd.DataFrame(
    [
        {
            "subject": "Glucose readings",
            "start": glucose_bounds[0],
            "end": glucose_bounds[1],
        },
        {
            "subject": "Workout records",
            "start": workout_bounds[0],
            "end": workout_bounds[1],
        },
    ]
)
summary

,subject,start,end
0,Glucose readings,2025-08-16 13:20:00,2025-08-30 17:54:00
1,Workout records,NaT,NaT


## Daily Compliance KPIs
We will blend SQL aggregates sourced from `glucose_readings` with pandas enrichment to calculate time-in-range (TIR) performance, volatility, and rolling trends.

In [ ]:
DAILY_COMPLIANCE_SQL = """
WITH base AS (
    SELECT
        DATE(timestamp) AS day,
        COUNT(*) AS reading_count,
        AVG(glucose_value) AS avg_glucose,
        MIN(glucose_value) AS min_glucose,
        MAX(glucose_value) AS max_glucose,
        SUM(CASE WHEN glucose_value BETWEEN 70 AND 180 THEN 1 ELSE 0 END) AS tir_count,
        SUM(CASE WHEN glucose_value < 70 THEN 1 ELSE 0 END) AS below_range_count,
        SUM(CASE WHEN glucose_value > 180 THEN 1 ELSE 0 END) AS above_range_count
    FROM glucose_readings
    WHERE timestamp BETWEEN ? AND ?
    GROUP BY DATE(timestamp)
)
SELECT
    day AS date,
    reading_count,
    avg_glucose,
    min_glucose,
    max_glucose,
    tir_count,
    below_range_count,
    above_range_count,
    ROUND(tir_count * 100.0 / NULLIF(reading_count, 0), 2) AS time_in_range_pct,
    ROUND(below_range_count * 100.0 / NULLIF(reading_count, 0), 2) AS time_below_range_pct,
    ROUND(above_range_count * 100.0 / NULLIF(reading_count, 0), 2) AS time_above_range_pct
FROM base
ORDER BY date;
"""


def load_daily_compliance(
    start: Optional[pd.Timestamp] = None,
    end: Optional[pd.Timestamp] = None,
    tir_target: float = 70.0,
) -> pd.DataFrame:
    min_date, max_date = get_date_range("glucose_readings", "timestamp")
    if min_date is None or max_date is None:
        raise ValueError("No glucose data available for compliance metrics.")

    start_ts = pd.to_datetime(start or min_date).floor("D")
    end_ts = pd.to_datetime(end or max_date).ceil("D") - pd.Timedelta(seconds=1)

    params = (start_ts.isoformat(), end_ts.isoformat())
    df = read_query_df(DAILY_COMPLIANCE_SQL, params=params)
    if df.empty:
        return df

    df["date"] = pd.to_datetime(df["date"])
    numeric_cols = [
        "reading_count",
        "avg_glucose",
        "min_glucose",
        "max_glucose",
        "tir_count",
        "below_range_count",
        "above_range_count",
        "time_in_range_pct",
        "time_below_range_pct",
        "time_above_range_pct",
    ]
    df[numeric_cols] = df[numeric_cols].apply(pd.to_numeric, errors="coerce")
    df = df.sort_values("date").reset_index(drop=True)

    df["rolling_7d_tir_pct"] = (
        df["time_in_range_pct"].rolling(window=7, min_periods=3).mean()
    )
    df["reading_count_ma"] = df["reading_count"].rolling(window=7, min_periods=3).mean()
    df["tir_target_met"] = np.where(
        df["time_in_range_pct"] >= tir_target, "On Target", "Needs Attention"
    )
    df["days_since_start"] = (df["date"] - df["date"].min()).dt.days
    return df

In [7]:
daily_kpis = load_daily_compliance()
daily_kpis.tail()

,date,reading_count,avg_glucose,min_glucose,max_glucose,tir_count,below_range_count,above_range_count,time_in_range_pct,time_below_range_pct,time_above_range_pct,rolling_7d_tir_pct,reading_count_ma,tir_target_met,days_since_start
9,2025-08-26,291,84.61,69.0,126.0,290,1,0,99.66,0.34,0.0,98.54,290.71,On Target,9
10,2025-08-27,289,90.21,67.0,138.0,288,1,0,99.65,0.35,0.0,98.49,290.86,On Target,10
11,2025-08-28,288,91.21,73.0,127.0,288,0,0,100.00,0.00,0.0,98.49,290.29,On Target,11
12,2025-08-29,292,91.76,59.0,132.0,287,5,0,98.29,1.71,0.0,98.24,290.57,On Target,12
13,2025-08-30,215,95.87,64.0,124.0,214,1,0,99.53,0.47,0.0,98.17,279.86,On Target,13


In [ ]:
if daily_kpis.empty:
    raise RuntimeError(
        "No daily KPI records were produced—verify glucose data coverage."
    )

fig_daily_overview = make_subplots(specs=[[{"secondary_y": True}]])
fig_daily_overview.add_trace(
    go.Scatter(
        x=daily_kpis["date"],
        y=daily_kpis["time_in_range_pct"],
        name="Time in Range %",
        mode="lines+markers",
        hovertemplate="%{x|%Y-%m-%d}: %{y:.1f}%<extra></extra>",
        line=dict(color="#1f77b4", width=2),
    ),
    secondary_y=False,
)
fig_daily_overview.add_trace(
    go.Scatter(
        x=daily_kpis["date"],
        y=daily_kpis["rolling_7d_tir_pct"],
        name="7-day Avg",
        mode="lines",
        line=dict(color="#ff7f0e", width=3, dash="dash"),
        hovertemplate="7d avg: %{y:.1f}%<extra></extra>",
    ),
    secondary_y=False,
)
fig_daily_overview.add_trace(
    go.Bar(
        x=daily_kpis["date"],
        y=daily_kpis["reading_count"],
        name="Readings per day",
        marker_color="rgba(31, 119, 180, 0.3)",
        hovertemplate="%{x|%Y-%m-%d}: %{y} readings<extra></extra>",
    ),
    secondary_y=True,
)
fig_daily_overview.add_hline(
    y=70,
    line=dict(color="green", dash="dot"),
    annotation_text="TIR Target 70%",
    annotation_position="top left",
)
fig_daily_overview.update_yaxes(
    title_text="Time in Range (%)", secondary_y=False, range=[0, 100]
)
fig_daily_overview.update_yaxes(title_text="Reading Count", secondary_y=True)
fig_daily_overview.update_layout(
    title="Daily Time-in-Range Compliance",
    legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1),
    hovermode="x unified",
    bargap=0.1,
    height=600,
)
fig_daily_overview.show()

In [ ]:
latest = daily_kpis.iloc[-1]
rolling_ref = daily_kpis["rolling_7d_tir_pct"].iloc[-1]

fig_tir_gauge = go.Figure(
    go.Indicator(
        mode="gauge+number+delta",
        value=latest["time_in_range_pct"],
        delta={"reference": rolling_ref, "valueformat": ".1f", "suffix": " pts"},
        gauge={
            "axis": {"range": [0, 100]},
            "bar": {"color": "#1f77b4"},
            "steps": [
                {"range": [0, 60], "color": "#f5b7b1"},
                {"range": [60, 70], "color": "#fcf3cf"},
                {"range": [70, 100], "color": "#d4efdf"},
            ],
            "threshold": {"line": {"color": "green", "width": 4}, "value": 70},
        },
        title={"text": f"Latest TIR ({latest['date'].date()})"},
        number={"suffix": "%", "valueformat": ".1f"},
    )
)
fig_tir_gauge.update_layout(height=350)
fig_tir_gauge.show()

## Workout-Linked Glucose Shifts
Leverage the `workout_glucose_impact` view to compare average glucose in the 30 minutes pre-workout vs. the 2 hours post-workout, then shape the results in pandas.

In [ ]:
WORKOUT_IMPACT_SQL = """
SELECT
    workout_type,
    workout_start,
    duration,
    total_energy_burned,
    pre_workout_glucose,
    post_workout_glucose
FROM workout_glucose_impact
WHERE workout_start BETWEEN ? AND ?
ORDER BY workout_start
;
"""


def load_workout_impacts(
    start: Optional[pd.Timestamp] = None,
    end: Optional[pd.Timestamp] = None,
    min_samples: int = 3,
) -> tuple[pd.DataFrame, pd.DataFrame]:
    overall_start, overall_end = get_date_range("glucose_readings", "timestamp")
    if overall_start is None or overall_end is None:
        raise ValueError("Glucose coverage is required to evaluate workout impacts.")

    workout_start = pd.to_datetime(start or overall_start).floor("D")
    workout_end = pd.to_datetime(end or overall_end).ceil("D") - pd.Timedelta(seconds=1)
    params = (workout_start.isoformat(), workout_end.isoformat())

    df = read_query_df(WORKOUT_IMPACT_SQL, params=params)
    if df.empty:
        return df, pd.DataFrame()

    df["workout_start"] = pd.to_datetime(df["workout_start"])
    df["duration"] = pd.to_numeric(df["duration"], errors="coerce")
    df["total_energy_burned"] = pd.to_numeric(
        df["total_energy_burned"], errors="coerce"
    )
    df["pre_workout_glucose"] = pd.to_numeric(
        df["pre_workout_glucose"], errors="coerce"
    )
    df["post_workout_glucose"] = pd.to_numeric(
        df["post_workout_glucose"], errors="coerce"
    )
    df["glucose_delta"] = df["post_workout_glucose"] - df["pre_workout_glucose"]
    df = df.dropna(subset=["pre_workout_glucose", "post_workout_glucose"])
    df = df.sort_values("workout_start").reset_index(drop=True)

    aggregated = (
        df.groupby("workout_type")
        .agg(
            workout_sessions=("workout_start", "count"),
            avg_duration_min=("duration", "mean"),
            avg_energy_burned=("total_energy_burned", "mean"),
            avg_delta=("glucose_delta", "mean"),
            avg_pre=("pre_workout_glucose", "mean"),
            avg_post=("post_workout_glucose", "mean"),
        )
        .query("workout_sessions >= @min_samples")
        .reset_index()
    )

    return df, aggregated

In [11]:
workout_events, workout_summary = load_workout_impacts()
(
    workout_events.head()
    if isinstance(workout_events, pd.DataFrame) and not workout_events.empty
    else "No workout events found"
)

'No workout events found'

In [12]:
if isinstance(workout_events, pd.DataFrame) and workout_events.empty:
    print("⚠️ No workouts with glucose coverage found in the selected window.")
else:
    fig_workout_scatter = px.scatter(
        workout_events,
        x="workout_start",
        y="glucose_delta",
        color="workout_type",
        size="duration",
        hover_data={
            "pre_workout_glucose": ":.1f",
            "post_workout_glucose": ":.1f",
            "duration": ":.1f",
            "total_energy_burned": ":.1f",
        },
        labels={"glucose_delta": "Δ Glucose (post - pre)"},
        title="Glucose Shift Around Workouts",
    )
    fig_workout_scatter.update_layout(
        height=500,
        legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1),
    )
    fig_workout_scatter.add_hline(y=0, line_dash="dot", line_color="gray")
    fig_workout_scatter.show()

    if isinstance(workout_summary, pd.DataFrame) and not workout_summary.empty:
        fig_workout_summary = px.bar(
            workout_summary.sort_values("avg_delta"),
            x="workout_type",
            y="avg_delta",
            color="avg_delta",
            color_continuous_scale="RdBu",
            labels={"avg_delta": "Average Δ Glucose (mg/dL)"},
            title="Average Glucose Change by Workout Type",
            text=workout_summary["workout_sessions"].astype(str) + " sessions",
        )
        fig_workout_summary.update_traces(textposition="outside")
        fig_workout_summary.update_layout(height=450, coloraxis_showscale=False)
        fig_workout_summary.add_hline(y=0, line_dash="dot", line_color="gray")
        fig_workout_summary.show()
    else:
        print("ℹ️ Not enough repeated workout types to summarize.")

⚠️ No workouts with glucose coverage found in the selected window.


## Export Artifacts
Persist tidy tables and dashboards to the `outputs/` directory so they can be shared without rerunning the notebook.

In [13]:
outputs_dir = project_root / "outputs"
outputs_dir.mkdir(exist_ok=True)

daily_csv_path = outputs_dir / "daily_compliance_kpis.csv"
workout_csv_path = outputs_dir / "workout_glucose_impacts.csv"
workout_summary_csv_path = outputs_dir / "workout_glucose_type_summary.csv"
dashboard_daily_path = outputs_dir / "daily_compliance_overview.html"
dashboard_workout_path = outputs_dir / "workout_glucose_shift.html"
dashboard_workout_type_path = outputs_dir / "workout_glucose_type_summary.html"

daily_kpis.to_csv(daily_csv_path, index=False)
print(f"💾 Saved daily KPIs to {daily_csv_path}")

if isinstance(workout_events, pd.DataFrame) and not workout_events.empty:
    workout_events.to_csv(workout_csv_path, index=False)
    print(f"💾 Saved workout event impacts to {workout_csv_path}")

    if isinstance(workout_summary, pd.DataFrame) and not workout_summary.empty:
        workout_summary.to_csv(workout_summary_csv_path, index=False)
        print(f"💾 Saved workout summary to {workout_summary_csv_path}")

    if "fig_workout_scatter" in globals():
        fig_workout_scatter.write_html(dashboard_workout_path, include_plotlyjs="cdn")
        print(f"🖼️ Exported workout dashboard to {dashboard_workout_path}")

    if "fig_workout_summary" in globals():
        fig_workout_summary.write_html(
            dashboard_workout_type_path, include_plotlyjs="cdn"
        )
        print(f"🖼️ Exported workout type dashboard to {dashboard_workout_type_path}")
else:
    print("ℹ️ No workout impact data available to export.")

if "fig_daily_overview" in globals():
    fig_daily_overview.write_html(dashboard_daily_path, include_plotlyjs="cdn")
    print(f"🖼️ Exported daily compliance dashboard to {dashboard_daily_path}")

if "fig_tir_gauge" in globals():
    gauge_path = outputs_dir / "daily_tir_indicator.html"
    fig_tir_gauge.write_html(gauge_path, include_plotlyjs="cdn")
    print(f"🖼️ Exported TIR gauge to {gauge_path}")

💾 Saved daily KPIs to /Users/george/Library/Mobile Documents/com~apple~CloudDocs/Programming Projects/Apple-Health-DS/outputs/daily_compliance_kpis.csv
ℹ️ No workout impact data available to export.
🖼️ Exported daily compliance dashboard to /Users/george/Library/Mobile Documents/com~apple~CloudDocs/Programming Projects/Apple-Health-DS/outputs/daily_compliance_overview.html
🖼️ Exported TIR gauge to /Users/george/Library/Mobile Documents/com~apple~CloudDocs/Programming Projects/Apple-Health-DS/outputs/daily_tir_indicator.html


## Summary
- SQL views power parameterized KPI feeds for daily time-in-range coverage.
- Pandas layers smoothed compliance insights and target attainment classification.
- Plotly dashboards translate the analytics into investor- and clinician-friendly visuals.
- Artifacts are exported for downstream sharing and inclusion in portfolio materials.